<a href="https://colab.research.google.com/github/Rbeautelus1/Rbeautelus1/blob/main/Copy_of_BERT_V_ROBERTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import string
!pip install emoji
import emoji

from transformers import RobertaTokenizer
from datasets import Dataset

import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer
from sklearn.model_selection import train_test_split

from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
from torch.utils.data import DataLoader, Dataset

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Loading Dataset

In [ ]:
data1 = pd.read_csv("/content/Tweets.csv")
data2 = pd.read_csv("/content/stock_data.csv")




In [ ]:
data1.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
data2.head()

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1


Clean dataset 1

In [ ]:
import re
import string
import emoji
import spacy
import nltk
from nltk.corpus import stopwords

# Download the stopwords dataset
nltk.download('stopwords')

# Load the spaCy model once
nlp = spacy.load('en_core_web_sm')

# Load stopwords
stopwords_list = set(stopwords.words('english'))

# Function to remove emojis from text
def strip_emoji(text):
    return emoji.replace_emoji(text, replace='')

# Function to strip all entities (newline, emojis, mentions, links, non-ASCII characters)
def strip_all_entities(text):
    text = text.replace('\r', '').replace('\n', ' ').lower()  # Remove newlines and lowercase
    text = re.sub(r'[^\x00-\x7f]', r'', text)  # Remove non-ASCII characters
    banned_list = string.punctuation + 'Ã' + '±' + 'ã' + '¼' + 'â' + '»' + '§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)  # Remove banned characters
    return text

# Function to clean hashtags
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet))
    return " ".join(word.strip() for word in re.split('#|_', new_tweet))  # Remove hashtags in the middle

# Main function to apply all cleaning steps
def text_cleaning(text):
    # Lowercase the text and remove emojis
    text = strip_emoji(text.lower())

    # Remove mentions and URLs
    text = re.sub(r'@\w+', '', text)  # Remove @username
    text = re.sub(r'https?://\S+', '', text)  # Remove URLs

    # Remove digits and special characters
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters

    # Remove emojis, hashtags, and non-ASCII characters
    text = strip_all_entities(text)
    text = clean_hashtags(text)

    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stopwords_list)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Ensure consistent column names for both datasets
data1.rename(columns={'text': 'Text'}, inplace=True)
data2.rename(columns={'Text': 'Text'}, inplace=True)

data1.rename(columns={'airline_sentiment': 'Sentiment'}, inplace=True)
data2.rename(columns={'Sentiment': 'Sentiment'}, inplace=True)

# Apply text cleaning to your datasets
data1['Cleaned_text'] = data1['Text'].apply(text_cleaning)
data1['Cleaned_text'] = data1['Cleaned_text'].str.replace(r'\s+', ' ', regex=True).str.strip()

data2['Cleaned_text'] = data2['Text'].apply(text_cleaning)
data2['Cleaned_text'] = data2['Cleaned_text'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Check the result for both datasets
print("Cleaned data1 texts:")
for text in data1['Cleaned_text'].iloc[0:10]:  # Display the first 10 cleaned texts for data1
    print(text)

print("\nCleaned data2 texts:")
for text in data2['Cleaned_text'].iloc[0:10]:  # Display the first 10 cleaned texts for data2
    print(text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Cleaned data1 texts:
said
plus youve added commercials experience tacky
didnt today must mean need take another trip
really aggressive blast obnoxious entertainment guests faces amp little recourse
really big bad thing
seriously would pay flight seats didnt playing really bad thing flying va
yes nearly every time fly vx ear worm wont go away
really missed prime opportunity men without hats parody
well didntbut
amazing arrived hour early youre good

Cleaned data2 texts:
kickers watchlist xide tit soq pnk cpw bpz aj trade method method see prev posts
user aap movie return feageed indicator trades year awesome
user id afraid short amzn looking like nearmonopoly ebooks infrastructureasaservice
mnta
oi
pgnx
aap user current downtrend break otherwise shortterm correction medterm downtrend
mondays relative weakness nyx win tie tap ice int bmc aon c chk biib
goog ower trend line channel test volume support
aap watch tomorrow ong entry


In [ ]:
data1.head()

,tweet_id,Sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,Text,tweet_coord,tweet_created,tweet_location,user_timezone,Cleaned_text
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),said
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),plus youve added commercials experience tacky
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),didnt today must mean need take another trip
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),really aggressive blast obnoxious entertainmen...
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),really big bad thing


In [ ]:
data2.head()

,Text,Sentiment,Cleaned_text
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1,kickers watchlist xide tit soq pnk cpw bpz aj ...
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1,user aap movie return feageed indicator trades...
2,user I'd be afraid to short AMZN - they are lo...,1,user id afraid short amzn looking like nearmon...
3,MNTA Over 12.00,1,mnta
4,OI Over 21.37,1,oi


Merging Dataset

In [ ]:
# Keep only the necessary columns for sentiment analysis
data1_cleaned = data1[['Cleaned_text', 'Sentiment']]
data2_cleaned = data2[['Cleaned_text', 'Sentiment']]

In [ ]:
# Merge the two datasets
merged_data = pd.concat([data1_cleaned, data2_cleaned], ignore_index=True)

In [ ]:
# Check the merged dataset
print(merged_data.head())

                                        Cleaned_text Sentiment
0                                               said   neutral
1      plus youve added commercials experience tacky  positive
2       didnt today must mean need take another trip   neutral
3  really aggressive blast obnoxious entertainmen...  negative
4                               really big bad thing  negative


In [ ]:
# Check for missing values
missing_values = merged_data.isna().sum()
print("Missing Values:\n", missing_values)


Missing Values:
 Cleaned_text    0
Sentiment       0
dtype: int64


In [ ]:
print(merged_data.dtypes)


Cleaned_text    object
Sentiment       object
dtype: object


In [ ]:
print(merged_data['Sentiment'].value_counts())


Sentiment
negative    9178
1           3685
neutral     3099
positive    2363
-1          2106
Name: count, dtype: int64


In [ ]:
merged_data.to_csv('cleaned_data.csv', index=False)


In [ ]:
loaded_data = pd.read_csv('cleaned_data.csv')
print(loaded_data.head())


                                        Cleaned_text Sentiment
0                                               said   neutral
1      plus youve added commercials experience tacky  positive
2       didnt today must mean need take another trip   neutral
3  really aggressive blast obnoxious entertainmen...  negative
4                               really big bad thing  negative


# ROBERTA Model

In [ ]:
merged_data = pd.read_csv('cleaned_data.csv')

Prepare the Data

In [ ]:
! pip install --upgrade datasets
from datasets import Dataset


import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

# Encode the sentiment labels
label_map = {'positive': 0, 'negative': 1, 'neutral': 2}  # Adjust according to your labels
merged_data['label'] = merged_data['Sentiment'].map(label_map)

# Split into training and validation sets
train_df, val_df = train_test_split(merged_data, test_size=0.2, random_state=42)

# Create datasets
train_dataset = Dataset.from_pandas(train_df[['Cleaned_text', 'label']])
val_dataset = Dataset.from_pandas(val_df[['Cleaned_text', 'label']])

Load the tokenizer and Model

In [ ]:
print(train_dataset)
print(val_dataset)


Dataset({
    features: ['Cleaned_text', 'label', '__index_level_0__'],
    num_rows: 16344
})
Dataset({
    features: ['Cleaned_text', 'label', '__index_level_0__'],
    num_rows: 4087
})


In [ ]:
print(train_dataset[2])  # Print the first entry to see its structure
print(val_dataset[0])    # Same for validation dataset
print(train_dataset)

{'Cleaned_text': 'flight phi den staffed great crew freecomedyshow newlifetimecustomer', 'label': 0.0, '__index_level_0__': 6383}
{'Cleaned_text': 'abbott laboratories takes ride coronavirus crisis happens make three', 'label': None, '__index_level_0__': 19631}
Dataset({
    features: ['Cleaned_text', 'label', '__index_level_0__'],
    num_rows: 16344
})


In [ ]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['Cleaned_text'], padding='max_length', truncation=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from transformers import RobertaTokenizer
from datasets import Dataset
import pandas as pd


# Convert DataFrames to Hugging Face Datasets

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['Cleaned_text'], padding='max_length', truncation=True)


# Tokenize the datasets

'''
train_dataset = pd.DataFrame(train_dataset)
val_dataset= pd.DataFrame(val_dataset)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

'''

#tokenized_train = train_dataset.map(tokenize_function, batched=True)
#tokenized_val = val_dataset.map(tokenize_function, batched=True)

tokenized_train = train_df.map(tokenize_function)
tokenized_val = val_df.map(tokenize_function)


# Rename label column to labels
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_val = tokenized_val.rename_column("label", "labels")

# Set the format of the datasets for PyTorch or TensorFlow
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Check the first entry of the tokenized train dataset
#print(tokenized_train[0])
print(tokenized_val[0])



TypeError: string indices must be integers

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
train_dataset = set([x['input_ids'] for x in tokenized_train])
val_dataset = set([x['input_ids'] for x in tokenized_val])
overlap = train_dataset.intersection(val_dataset)
print(f"Number of overlapping instances: {len(overlap)}")


Number of overlapping instances: 0


Training the Model

In [ ]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)


# Define your training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=8,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,
    evaluation_strategy = "epoch",   # batch size for evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir='./logs',
    do_train = True,
    do_eval = True, # directory for storing logs

)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics = compute_metrics,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

# Start training
trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.700719,0.500000,0.000000,0.000000,0.000000
2,No log,0.700676,0.500000,0.000000,0.000000,0.000000
3,No log,0.700591,0.500000,0.000000,0.000000,0.000000
4,No log,0.700470,0.500000,0.000000,0.000000,0.000000
5,No log,0.700307,0.500000,0.000000,0.000000,0.000000
6,No log,0.700103,0.500000,0.000000,0.000000,0.000000
7,No log,0.699861,0.500000,0.000000,0.000000,0.000000
8,No log,0.699577,0.500000,0.000000,0.000000,0.000000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

TrainOutput(global_step=8, training_loss=0.6941356062889099, metrics={'train_runtime': 182.8413, 'train_samples_per_second': 0.088, 'train_steps_per_second': 0.044, 'total_flos': 4209776885760.0, 'train_loss': 0.6941356062889099, 'epoch': 8.0})

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 0.6995770931243896, 'eval_accuracy': 0.5, 'eval_f1': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_runtime': 4.6322, 'eval_samples_per_second': 0.432, 'eval_steps_per_second': 0.216, 'epoch': 8.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
trainer.save_model("./final_model")


# BERT Model

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Adjust num_labels as needed


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenize the dataset

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['Cleaned_text'], padding='max_length', truncation=True)

# Assuming you have train_dataset and val_dataset defined
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)


AttributeError: 'set' object has no attribute 'map'

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Training Arguments

In [ ]:

# Define your training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,
    evaluation_strategy = "epoch",   # batch size for evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    do_train = True,
    do_eval = True, # directory for storing logs

)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics = compute_metrics,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)







Train model

In [ ]:
#train the model
trainer.train()

#evaluate the model
trainer.evaluate()



IndexError: index out of range in self

In [ ]:
model.save_pretrained('./bert_model')
tokenizer.save_pretrained('./bert_model')


('./bert_model/tokenizer_config.json',
 './bert_model/special_tokens_map.json',
 './bert_model/vocab.txt',
 './bert_model/added_tokens.json')

In [ ]:
git remote add origin https://github.com/AI-Diversity-Research/NLP-sentimentAnalysis
git branch -M main
git push -u origin main


SyntaxError: invalid syntax (<ipython-input-49-e94c4bb5c3de>, line 1)